<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license.  
(c) Nicole Beisiegel</td>
</table>

In [2]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

from write_hdf5 import *
from write_par import *
from CreateSpectra import *
from bathymetry import *

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# A High-Order Spectral Model for 3D Water Wave Simulations
## Initial Conditions

This notebook creates user-defined initial conditions for the **HOS** model. Two files in ``HDF5`` format will be created:

- initdata_``<TestName>``.1.h5, a file that contains the initial potential, bathymetry and surface elevation,
- initpars_``<TestName>``.h5, a file that contains physical and numerical parameters,

where ``<TestName>`` is a user-specified indicator.


To get more familiar with the model, please check out and run the _testsuite_. Information on the current version of the _testsuite_ with an overview over available post-processing routines can be found in the **iPython notebook** [here.](HOS_Visualization.ipynb)  
Please, before you send a PR (pull request) @beisiegel on **GitHub**, run the _testsuite_ to ensure that all features are still running with your changes.

This script produces initial conditions in `HDF5` format for the **HOS** model and plots the initial sea surface elevation $\eta$, the velocity potential $\phi_{tot}$ and the bathymetry $b$ for time $t=0$.

Current options are for the ..

..**SPECTRUM**: 

- "``JONSWAP``": A Jonswap spectrum (Please specify Jonswap parameters below by editing ``JonswapParam``.)
- "``RF``": Stream function solution of Rienecker & Fenton (1981)
- "``STOKES``": Essentially 2D version of second order approximation of Stokes wave train (see Craig & Sulem, 1993)
- "``CONSTANT``": A constant water height (for wellbalancing tests)
- "``SINE``": A single sinusoidal wave and a linear background velocity

..**BATHYMETRY**:

- "``zero``": No bathymetry
- "``constant``": Bathymetry that is constant (=base) everywhere, specify value for base in ``bathyParam``
- "``linear``": A linear sloping bathymetry with base (=base) and slope (=slope) to be specified in ``bathyParam``
- "``elliptical``": An elliptical mould around a point ($x_0, y_0$) to be specified in ``bathyParam``
- "``hill``": A piecewise linear (between $x_1, x_2, x_3$ and $x_4$) hill, constant in $y$-direction in the middle of the domain to be specified in ``bathyParam`` 

To further specify the initial conditions, please adjust the corresponding parameters in the box below.

In [5]:
#------------------------------------------------------------------
#-- To be altered by user

SimFolder = '/home/nicole/Development/hos/data/'
TestName  = 'wavemaker'

#-- You choose (type of spectrum and bathymetry):
spectrum = "Stokes"
bathymetry = "zero"

# Standard configurations for parameter files:
JonswapParam = {'g': 9.81, 'gamma': 3.3, 'alpha_p': 0.081, 'dir_spr': 40.*np.pi/180., 'kp': 0.033, \
                'lmbda': 2*np.pi/0.033,'omega_p': np.sqrt(9.81*0.033), 'epsilon': 0.12, 'theta_p': 0.0,\
                'h': 1000., 'sigma': 0.06/0.033, 'hs': (0.24/0.033)}

StokesParam ={'a': 0.065, 'k': 2.50, 'h': 100.0, 'g': 9.81} #9.81 k=5 h=10

RFParam = {'H': .04, 'c': 1.0, 'tau': 40.0, 'cE': 0.0, 'g': 9.81, 'h': 100.0}
           # H wave height, c wave speed, tau period of wave, cE current speed in Eulerian frame
    
Param = {'h': 10.0, 'g': 9.81} #{'h': 45.72, 'g': 9.81}

SineParam = {'a': 1.0, 'b1': 0.10, 'b2': 5.0, 'c': 0.0, 'x0': 5.0, 'h': 0.250, 'g': 9.81}

#-- Set your parameters here
#parameters = JonswapParam  
parameters = StokesParam
#parameters = RFParam
#parameters = Param
#parameters = SineParam

#-- Set bathymetry config here
bathyParam = {'base': .000, 'slope': 0.01, 'x0': 15.0, 'y0': 10.0, 'x1': 16.0, 'x2': 22.0, 'x3': 24.0, 'x4': 27.0}
#bathyParam = {'base': .000, 'slope': 0.01, 'x0': 6.1, 'y0': 13.72, 'x1': 16.0, 'x2': 22.0, 'x3': 24.0, 'x4': 27.0}
    
#-- Labels for the simulation run
saveflg  = 1
runsubid = 1

#-- Relaxation zones on=True/off=False; relax_* is the length of the relaxation zone in *-direction (on both sides)
Relax = 1

#-- Relaxation zone switch on/off
relax = True

#-- Discretization: Number of cells (fourier modes) in x and y direction
alpha = 8
Nx    = 2**alpha  
Ny    = Nx

#-- Length of the domain
#-- For JONSWAP choose something like 32*parameters['lmbda'] 
#-- For Stokes [0,2\pi]
#-- For elliptical bathy Lx = 25, Ly = 30

Lx = 10*np.pi # 100# 32*parameters['lmbda'] #25. #2*np.pi #25
Ly = 2*np.pi #10.0 #Lx #30. #Lx

Lx1 = 0.0
Ly1 = 0.0
Lx2 = Lx
Ly2 = Ly

if (relax): 
    Relax = 1
    Lx1 = 8*np.pi
    Ly1 = 0.00 #0.005; np.pi

    Lx += 2.*Lx1
    Ly += 2.*Ly1
    
    Lx2 = Lx - Lx1
    Ly2 = Ly - Ly1

#-- Time parameters
dt     = 0.0125

Tp     = 1.0  
T      = 5* Tp   
dtsave = 1.0

#-- Filtering Parameter
Tramp  = 200

#------------------------------------------------------------------
#-- Only internal subroutines follow

#-- Wavenumber domain
kx = 2*np.pi/Lx*np.arange(-Nx/2,Nx/2)
ky = 2*np.pi/Ly*np.arange(-Ny/2,Ny/2)

lx = np.linspace(0,Lx,Nx,endpoint=True) 
ly = np.linspace(0,Ly,Ny,endpoint=True) 

#-- The options for the different spectra
switch = {'JONSWAP':  create_jonswap,
          'RF':       create_rieneckerfenton,
          'Gauss':    create_gauss,
          'Stokes':   create_stokes,
          'CONSTANT': create_constant,
          'Sine':     create_sine}

if spectrum in switch:
     etar, phir = switch[spectrum](kx,ky,lx,ly,Param = parameters)
else:
     pass

bathy = create_bathymetry(lx,ly, Type = bathymetry, Param = bathyParam)

#-- Plot initial condition
plt.figure(1)
plt.suptitle("Initial Condition TYPE: "+str(spectrum),fontsize=18)

plt.subplot(221)
plt.pcolormesh(lx,ly,etar, cmap=plt.get_cmap("Blues"))
plt.colorbar()
plt.title("Initial Sea Surface Elevation $\eta$")
plt.xlabel("x")
plt.xlim(np.amin(lx), np.amax(lx))
plt.ylabel("y")
plt.ylim(np.amin(ly), np.amax(ly))

plt.subplot(222)
plt.pcolormesh(lx,ly,bathy) #, cmap=plt.get_cmap("Greys"))
plt.colorbar()    
plt.title("Bathymetry $b$")
plt.xlabel("x")
plt.xlim(0.0, Lx)
plt.ylabel("y")
plt.ylim(0.0, Ly)

plt.subplot(223)
plt.pcolormesh(lx,ly,phir, cmap=plt.get_cmap("Greens"))
plt.colorbar()
plt.title("Initial Velocity Potential $\phi_{tot}$")
plt.xlabel("x")
plt.xlim(np.amin(lx), np.amax(lx))
plt.ylabel("y")
plt.ylim(np.amin(ly), np.amax(ly))

plt.tight_layout()
plt.show()

plt.close()

#-- Write information
write_hdf5(SimFolder, TestName, Nx, Ny, Lx, Ly, parameters['g'], parameters['h'], dt, T, dtsave, saveflg, runsubid, \
           Tramp, Relax, Lx1, Lx2, Ly1, Ly2, etar, phir, bathy) 
write_par(SimFolder, TestName, Nx, Ny, Lx, Ly, parameters['g'], parameters['h'], dt, T, dtsave, saveflg, runsubid, \
           Tramp, Relax, Lx1, Lx2, Ly1, Ly2) 

Success!
Output files written.
Success!
Parameter file written.
